In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm 
from time import sleep

**собираем названия всех региональных центров с 2-ух источников**

In [2]:
# собираем названия регионалльных центров с 1-ого источника: https://marketing-course.ru/administrativnye-centry-regionov/
regional_centers_1 = set() # множество, т.к. у некоторых различных регионов одинаковые центры 
my_response = requests.get('https://marketing-course.ru/administrativnye-centry-regionov/').text
soup = BeautifulSoup(my_response, 'lxml')
table = soup.find_all('table')[0]
list_regions_info = table.find_all('tr')[1:]
for region in list_regions_info:
    title_regional_center = region.find_all('td')[3].text
    regional_centers_1.add(title_regional_center)
print(regional_centers_1, end="\n\n")
print(len(regional_centers_1))

{'Улан-Удэ', 'Брянск', 'Калуга', 'Псков', 'Москва', 'Вологда', 'Волгоград', 'Тверь', 'Махачкала', 'Нижний Новгород', 'Тула', 'Липецк', 'Барнаул', 'Чита', 'Петропавловск-Камчатский', 'Южно-Сахалинск', 'Ульяновск', 'Черкесск', 'Тюмень', 'Владивосток', 'Тамбов', 'Ярославль', 'Кострома', 'Екатеринбург', 'Майкоп', 'Элиста', 'Томск', 'Нальчик', 'Курск', 'Владивкавказ', 'Самара', 'Ростов', 'Саранск', 'Иваново', 'Ставрополь', 'Рязань', 'Краснодар', 'Новосибирск', 'Омск', 'Сыктывкар', 'Челябиннск', 'Магадан', 'Горно-Алтайск', 'Смоленск', 'Мурманск', 'Оренбург', 'Магас', 'Воронеж', 'Биробиджан', 'Астрахань', 'Курган', 'Благовещенск', 'Пермь', 'Владимир', 'Ижевск', 'Анадырь', 'Казань', 'Чебоксары', 'Петрозаводск', 'Санкт-Петербург', 'Кызыл', 'Архангельск', 'Кемерово', 'Салехард', 'Грозный', 'Ханты-мансийск', 'Красноярск', 'Хабаровск', 'Якутск', 'Орел', 'Пенза', 'Йошкар-Ола', 'Киров', 'Уфа', 'Белгород', 'Великий Новгород', 'Саратов', 'Иркутск', 'Калининград', 'Абакан'}

80


In [3]:
# собираем названия регионалльных центров со 2-ого источника: https://se33.ru/2-uncategorised/378-regions.html
regional_centers_2 = set() 
my_response = requests.get('https://se33.ru/2-uncategorised/378-regions.html').text
soup = BeautifulSoup(my_response, 'lxml')
table = soup.find('table')
list_regions_info = table.find_all('tr')[1:]
for region in list_regions_info:
    title_regional_center = region.find_all('td')[3].text
    regional_centers_2.add(title_regional_center)
print(regional_centers_2, end="\n\n")
print(len(regional_centers_2))

{'Улан-Удэ', 'Брянск', 'Калуга', 'Псков', 'Ростов-на-Дону', 'Волгоград', 'Вологда', 'Москва', 'Тверь', 'Махачкала', 'Нижний Новгород', 'Тула', 'Липецк', 'Барнаул', 'Петропавловск-Камчатский', 'Чита', 'Южно-Сахалинск', 'Черкесск', 'Ульяновск', 'Тюмень', 'Владивосток', 'Тамбов', 'Ярославль', 'Кострома', 'Екатеринбург', 'Кудымкар', 'Элиста', 'Майкоп', 'Томск', 'Нальчик', 'Курск', 'Самара', 'Саранск', 'Ставрополь', 'пгт Агинское', 'Рязань', 'Краснодар', 'Абакан', 'Новосибирск', 'Омск', 'Сыктывкар', 'пгт Усть-Ордынский', 'Магадан', 'Горно-Алтайск', 'Назрань', 'Новгород', 'Смоленск', 'Мурманск', 'Оренбург', 'Воронеж', 'пгт Тура', 'Ханты-Мансийск', 'Челябинск', 'Биробиджан', 'Астрахань', 'Курган', 'Нарьян-Мар', 'Владикавказ', 'Благовещенск', 'Дудинка', 'Пермь', 'Владимир', 'Ижевск', 'Анадырь', 'Казань', 'Чебоксары', 'Петрозаводск', 'Кызыл', 'Санкт-Петербург', 'Архангельск', 'Кемерово', 'Грозный', 'Салехард', 'Красноярск', 'Хабаровск', 'пгт Палана', 'Якутск', 'Орел', 'Пенза', 'Йошкар-Ола', 'Ки

**объединяем оба множества (union)**

In [4]:
all_regional_centers = regional_centers_1.union(regional_centers_2)
print(all_regional_centers, end='\n\n')
print(len(all_regional_centers))

{'Улан-Удэ', 'Брянск', 'Калуга', 'Псков', 'Ростов-на-Дону', 'Москва', 'Вологда', 'Волгоград', 'Тверь', 'Махачкала', 'Нижний Новгород', 'Тула', 'Липецк', 'Барнаул', 'Чита', 'Петропавловск-Камчатский', 'Южно-Сахалинск', 'Ульяновск', 'Черкесск', 'Тюмень', 'Владивосток', 'Тамбов', 'Ярославль', 'Кострома', 'Екатеринбург', 'Кудымкар', 'Майкоп', 'Элиста', 'Томск', 'Нальчик', 'Курск', 'Владивкавказ', 'Самара', 'Ростов', 'Саранск', 'Ставрополь', 'Иваново', 'пгт Агинское', 'Рязань', 'Краснодар', 'Новосибирск', 'Омск', 'Сыктывкар', 'пгт Усть-Ордынский', 'Челябиннск', 'Магадан', 'Горно-Алтайск', 'Назрань', 'Новгород', 'Смоленск', 'Мурманск', 'Оренбург', 'Магас', 'Воронеж', 'пгт Тура', 'Ханты-Мансийск', 'Челябинск', 'Биробиджан', 'Астрахань', 'Курган', 'Нарьян-Мар', 'Владикавказ', 'Благовещенск', 'Дудинка', 'Пермь', 'Владимир', 'Ижевск', 'Анадырь', 'Казань', 'Чебоксары', 'Петрозаводск', 'Санкт-Петербург', 'Кызыл', 'Архангельск', 'Кемерово', 'Салехард', 'Грозный', 'Ханты-мансийск', 'Красноярск', 'Ха

In [5]:
titles = []
adresses = []
websites = []
phone_numbers = []
emails = []

In [6]:
def input_info(): # поиск информации
    input_tab = browser.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div/input")
    input_tab.send_keys(regional_center)
    
def click_search_button(): # нажатие на кнопку поиска
    search_button = browser.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div/button[2]")
    search_button.click()

def click_clear_info_button(): # нажатие на кнопку удаления текста в области поиска
    clear_info_button = browser.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div/button[1]')
    clear_info_button.click()
    
    
def collect_data(): # сбор всех данных(адрес, сайт, телефон, почта) с найденного представителя поддержки бизнеса 
    try:
        title = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/div').text
        adress = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/table/tbody/tr[1]/td[2]').text
        website = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/table/tbody/tr[2]/td[2]/a').get_attribute('href')
        phone = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/table/tbody/tr[3]/td[2]').text
        email = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/table/tbody/tr[4]/td[2]/a').text
    except NoSuchElementException:
        title = None
        adress = None
        website = None
        phone = None
        email = None 
    titles.append(title)
    adresses.append(adress)
    websites.append(website)
    phone_numbers.append(phone)
    emails.append(email)

In [7]:
browser = Chrome()
url = 'https://xn--l1agf.xn--p1ai/services/reg-support-map/'
browser.maximize_window() 
browser.get(url)
sleep(1) # пауза, чтобы страницы успела прогрузиться 
for regional_center in all_regional_centers:
    input_info()
    click_search_button()
    sleep(0.8)
    collect_data()
    click_clear_info_button()

In [8]:
data = {'название': titles, 'адрес': adresses, 'сайт': websites, 'телефон': phone_numbers, 'почта': emails}
df = pd.DataFrame(data=data)

In [9]:
df

,название,адрес,сайт,телефон,почта
0,Гарантийный фонд Бурятии,"Респ Бурятия, г Улан-Удэ, ул Смолина, 65, 670000",http://msp03.ru/,+78003030123,info@msp03.ru
1,ГАУ БО ЦОУ «Мой бизнес»,"Брянская, Брянск, Бежицкая, 54, 241023",https://xn--32-9kcqjffxnf3b.xn--p1ai/,+7 (4832) 58-92-77,info@mybiz32.ru
2,"ГАУ КО ""Агентство развития бизнеса""","Калужская, Калуга, Театральная, 38А, 248000",https://arbko.ru/,+7 4842 20-73-37,arbko@adm.kaluga.ru
3,"АНО ""ФГРП ПО"" (МКК)","Псковская область, Псков, Гоголя , 14, 180000",https://msp60.ru/,+7 811 233-13-37,office@msp60.ru
4,"АНО МФК ""РРАПП""","Ростовская область, г. Ростов-на-Дону, ул. Сед...",http://www.rrapp.ru%20/%20mbrostov.ru,+7 863 204-19-11,info@rrapp.ru
...,...,...,...,...,...
88,"ГОАУ ""АРНО""","Новгородская обл, г Великий Новгород, ул Фёдор...",http://mb53.ru/,8 800 550 11 88,info@mb53.ru
89,"ООО ""Бизнес-инкубатор СО""","Саратовская, Саратов, Краевая, 85, 410012",https://saratov-bis.ru/,8-8452-24-54-78,office@saratov-bi.ru
90,Фонд Центр Мой бизнес,"Иркутская, Иркутск, Рабочая, 2а/4, 501, 664011",https://mb38.ru/,+7 395 220-21-02,info@fondirk.ru
91,ЦПП КО,"Калининградская область, Калининград, Уральска...",http://mbkaliningrad.ru/,+7 4012 99 49 04,info@mbkaliningrad.ru


In [10]:
postal_codes = []
regions = []
cities = []
streets = []

def add_information_to_lists(number_postal_code, name_region, name_sity, name_street):
    postal_codes.append(number_postal_code)
    regions.append(name_region)
    cities.append(name_sity)
    streets.append(name_street)

def get_postal_code(full_adress):
    return full_adress.split(',')[-1]

def get_region(full_adress):
    return full_adress.split(',')[0]

def get_city(full_adress):
    return full_adress.split(',')[1]

def get_street(full_adress):
    return full_adress.split(',')[2]

for full_adress in df['адрес']:
    if full_adress != None:
        postal_code = get_postal_code(full_adress)
        region = get_region(full_adress)
        city = get_city(full_adress)
        street = get_street(full_adress)
    else:
        postal_code = region = city = street = None 
    add_information_to_lists(postal_code, region, city, street)
        
print(postal_codes)
print(regions)
print(cities)
print(streets)

[' 670000', ' 241023', ' 248000', ' 180000', ' 344006', ' 105077', ' 160000', ' 400012', ' 170034', ' 367015', ' 603144', ' 300004', ' 398050', ' 656056', ' 672000', None, ' 693023', ' 432072', ' 369000', ' 625000', ' 690091', ' 392000', ' 150000', ' 156019', ' 620075', None, ' 385000', ' 358000', ' 634050', ' 360000', ' 305000', None, ' 443100', ' 344006', ' 430005', ' 355012', ' 153037', None, ' 390000', ' 350911', ' 630007', None, ' 167000', None, None, ' 685000', ' 649000', ' 386101', None, ' 214014', None, ' 460019', None, ' 394018', None, ' 628011', ' 454006', ' 679016', ' 414000', ' 640027', ' 166000', ' 362019', ' 675002', None, ' 614096', ' 600005', ' 426008', ' 689000', ' 420107', ' 428003', ' 185035', None, ' 667000', ' 163000', ' 650002', ' 629007', ' 364024', ' 628011', ' 660016', ' 680000', None, ' 677027', None, ' 440061', ' 424033', None, ' 450015', ' 308033', ' 173000', ' 410012', ' 664011', ' 236022', ' 655010']
['Респ Бурятия', 'Брянская', 'Калужская', 'Псковская обл

In [11]:
df['регион'] = regions
df['город'] = cities
df['улица'] = streets
df['почтовый индекс'] = postal_codes

In [12]:
df.head()

,название,адрес,сайт,телефон,почта,регион,город,улица,почтовый индекс
0,Гарантийный фонд Бурятии,"Респ Бурятия, г Улан-Удэ, ул Смолина, 65, 670000",http://msp03.ru/,+78003030123,info@msp03.ru,Респ Бурятия,г Улан-Удэ,ул Смолина,670000
1,ГАУ БО ЦОУ «Мой бизнес»,"Брянская, Брянск, Бежицкая, 54, 241023",https://xn--32-9kcqjffxnf3b.xn--p1ai/,+7 (4832) 58-92-77,info@mybiz32.ru,Брянская,Брянск,Бежицкая,241023
2,"ГАУ КО ""Агентство развития бизнеса""","Калужская, Калуга, Театральная, 38А, 248000",https://arbko.ru/,+7 4842 20-73-37,arbko@adm.kaluga.ru,Калужская,Калуга,Театральная,248000
3,"АНО ""ФГРП ПО"" (МКК)","Псковская область, Псков, Гоголя , 14, 180000",https://msp60.ru/,+7 811 233-13-37,office@msp60.ru,Псковская область,Псков,Гоголя,180000
4,"АНО МФК ""РРАПП""","Ростовская область, г. Ростов-на-Дону, ул. Сед...",http://www.rrapp.ru%20/%20mbrostov.ru,+7 863 204-19-11,info@rrapp.ru,Ростовская область,г. Ростов-на-Дону,ул. Седова,344006


In [13]:
df = df.dropna(subset=['название'])

In [14]:
df.to_excel("list_business_support_representatives.xlsx", index=False)